In [1]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        if len(token_ids) == max_length:
            input_chunk = token_ids
            target_chunk = token_ids[1:] + [token_ids[-1]]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
        elif len(token_ids) < max_length:
            padded_tokens = token_ids + [0] * (max_length - len(token_ids))
            input_chunk = padded_tokens
            target_chunk = padded_tokens[1:] + [0]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
        else:
            for i in range(0, len(token_ids) - max_length, stride):
                input_chunk = token_ids[i : i + max_length]
                target_chunk = token_ids[i + 1 : i + max_length + 1]
                self.input_ids.append(torch.tensor(input_chunk))
                self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=2, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    
    return dataloader    






In [2]:
import torch.nn as nn
class CausalAttention(nn.Module):
    
    def __init__(self, d_in, d_out, context_length, dropout):
        super().__init__()
        self.w_query = torch.nn.Linear(d_in, d_out, bias=False)
        self.w_key = torch.nn.Linear(d_in, d_out, bias=False)
        self.w_value = torch.nn.Linear(d_in, d_out, bias=False)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
        
    def forward(self, inputs):
        queries = self.w_query(inputs)
        keys = self.w_key(inputs)
        values = self.w_value(inputs)
        
        attn_scores = queries @ keys.transpose(-2, -1) 
        attn_scores = attn_scores.masked_fill(self.mask == 1, float('-inf'))
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        context_vec = attn_weights @ values
        
        return context_vec

In [3]:
import torch
import torch.nn as nn

class MultiHeadCausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads):
        super().__init__()
        assert d_out % num_heads == 0
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.heads = nn.ModuleList([
            CausalAttention(d_in, self.head_dim, context_length, dropout) 
            for _ in range(num_heads)
        ])

        self.out_proj = nn.Linear(d_out, d_out)
        
    def forward(self, inputs):
        head_outputs = [head(inputs) for head in self.heads]
        concatenated = torch.cat(head_outputs, dim=-1)
        output = self.out_proj(concatenated)
        return output

In [4]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.gelu = nn.GELU()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [5]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, context_length, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadCausalAttention(
            d_in=d_model, 
            d_out=d_model, 
            context_length=context_length, 
            dropout=dropout, 
            num_heads=num_heads
        )
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        attn_out = self.attention(x)
        x = self.norm1(x + self.dropout(attn_out))
        ff_out = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_out))
        
        return x

In [6]:
class GPTModel(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers, d_ff, context_length, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.context_length = context_length
        
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(context_length, d_model)
        
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff, context_length, dropout)
            for _ in range(num_layers)
        ])
        
        self.final_norm = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_ids):
        batch_size, seq_len = input_ids.shape
        token_embeddings = self.token_embedding(input_ids)
        pos_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0)
        pos_embeddings = self.pos_embedding(pos_ids)
        
        x = token_embeddings + pos_embeddings
        x = self.dropout(x)
        
        for block in self.transformer_blocks:
            x = block(x)
        
        x = self.final_norm(x)
        logits = self.lm_head(x)
        
        return logits

In [13]:
raw_text = "Yours journey start"
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = tokenizer.encode(raw_text, allowed_special={"<|endoftext|>"})
max_length = len(token_ids)
vocab_size = 50257
output_dim = 3

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
pos_embedding_layer = torch.nn.Embedding(max_length, output_dim)

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=max_length,
    stride=max_length, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

token_embeddings = token_embedding_layer(inputs)
pos_embeddings = pos_embedding_layer(torch.arange(max_length))

input_embeddings = token_embeddings + pos_embeddings
input_embeddings[0]




vocab_size = 50257  
d_model = 512       
num_heads = 8       
num_layers = 6      
d_ff = 2048        
context_length = len(token_ids)
dropout = 0.1

model = GPTModel(
    vocab_size=vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    context_length=context_length,
    dropout=dropout
)


dataloader = create_dataloader_v1(
    raw_text, 
    batch_size=1, 
    max_length=context_length,
    stride=context_length, 
    shuffle=False
)


data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print(f"Input shape: {inputs.shape}")
print(f"Input tokens: {inputs}")
print(f"Target tokens: {targets}")


model.eval()
with torch.no_grad():
    logits = model(inputs)

print(logits)    
print(f"Output logits shape: {logits.shape}")

predictions = torch.argmax(logits, dim=-1)
print(f"Predicted token IDs: {predictions}")

for i, pred_tokens in enumerate(predictions):
    decoded = tokenizer.decode(pred_tokens.tolist())
    print(f"Decoded prediction {i}: '{decoded}'")

Input shape: torch.Size([1, 4])
Input tokens: tensor([[  56, 4662, 7002,  923]])
Target tokens: tensor([[4662, 7002,  923,  923]])
tensor([[[ 1.3864, -0.0533, -0.0948,  ...,  0.8021,  0.6421,  0.0644],
         [-0.0508, -0.1429,  0.7952,  ...,  0.7350,  0.1501,  0.2361],
         [ 1.1689, -0.2618,  0.3253,  ..., -0.0448,  0.7914,  0.7151],
         [ 0.6454,  0.5646,  0.4797,  ..., -1.3333,  0.8063,  0.6064]]])
Output logits shape: torch.Size([1, 4, 50257])
Predicted token IDs: tensor([[47674, 19779, 44010,  9166]])
Decoded prediction 0: ' Particularly {" predicate 71'


## Calculate training validation

In [10]:
def calculate_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss

def calculate_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calculate_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_loss = calculate_loss_loader(dataloader, model, device)
print(f"Training loss: {train_loss:.4f}")

Training loss: 10.4189
